# Indexes

Table indexes are data structures that allow fast lookups by an indexed attribute or combination of attributes.

In DataJoint, indexes are created by one of the three mechanisms:

1. Primary key 
2. Foreign key 
3. Explicitly defined indexes

The first two mechanisms are obligatory. Every table has a primary key, which serves as an unique index. Therefore, restrictions by a primary key are very fast. Foreign keys create additional indexes unless a suitable index already exists.

Let's test this principle. Let's create a table with a 10,000 entries and compare lookup times:

In [1]:
import datajoint as dj
schema = dj.schema('test_indexes')
schema.drop() # drop previous schema definition (if any) and create anew
schema = dj.schema('test_indexes')

Connecting dimitri@localhost:3306
Proceed to delete entire schema `test_indexes`? [yes, No]: yes


Let's say a mouse in the lab has a lab-specific ID but it also has a separate id issued by the animal facility.

In [2]:
@schema
class Mouse(dj.Manual):
    definition = """
    mouse_id : int  # lab-specific ID
    ---
    tag_id : int  # animal facility ID
    """

In [3]:
import random
def populate_mice(n=200_000):
    """insert a bunch of mice"""
    Mouse.insert(
        ((random.randint(1,1000_000_000), random.randint(1,1000_000_000)) 
         for i in range(n)), skip_duplicates=True)

In [4]:
populate_mice()

In [5]:
Mouse()

mouse_id lab-specific ID,tag_id animal facility ID
11217,875281392
17552,152331807
20961,540276155
21556,412369250
27310,477066288
29424,431836526
32469,279566781
35496,479823219
36278,708425691
38345,257800827


In [6]:
%%timeit -n10 -r10

# efficient! Uses the primary key
(Mouse() & {'mouse_id': random.randint(0, 999_999)}).fetch()

3.35 ms ± 520 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [7]:
%%timeit -n10 -r10

# inefficient! Requires a full table scan
(Mouse() & {'tag_id': random.randint(0, 999_999)}).fetch()

33.8 ms ± 3.7 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


The indexed searches are much faster!

To make searches faster on fields other than the primary key or a foreign key, you can add a secondary index explicitly. 

Regular indexes are declared as `index(attr1, ..., attrN)` on a separate line anywhere in the table declration (below the primary key divide). 

Indexes can be declared with unique constraint as  `unique index (attr1, ..., attrN)`.

Let's redeclare the table with a unique index on `tag_id`.

In [8]:
Mouse.drop()

`test_indexes`.`mouse` (199976 tuples)
Proceed? [yes, No]: yes
Tables dropped.  Restart kernel.


In [9]:
@schema
class Mouse(dj.Manual):
    definition = """
    mouse_id : int  # lab-specific ID
    ---
    tag_id : int  # animal facility ID
    unique index (tag_id)
    """

In [10]:
populate_mice()

In [11]:
Mouse()

mouse_id lab-specific ID,tag_id animal facility ID
613235850,9035
851759450,23286
288526852,35612
334893028,36840
966899091,38743
975639724,41615
266160312,42392
631040577,47535
892660895,60853
393521703,66230


Now both types of searches are equally efficient!

In [12]:
%%timeit -n10 -r10

#efficient! Uses the primary key
(Mouse() & {'mouse_id': random.randint(0, 999_999)}).fetch()

3.01 ms ± 466 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [13]:
%%timeit -n10 -r10

#efficient! Uses the seconary index on tag_id
(Mouse() & {'tag_id': random.randint(0, 999_999)}).fetch()

3.26 ms ± 526 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


Let's now imagine that rats in the `Rat` table are identified by the combination of lab the `lab_name` and `rat_id` in each lab:

In [14]:
@schema
class Rat(dj.Manual):
    definition = """
    lab_name : char(16) 
    rat_id : int unsigned # lab-specific ID
    ---
    date_of_birth = null : date
    """

In [15]:
def populate_rats():
    lab_names = ("Cajal", "Kandel", "Moser", "Hubel & Wiesel")
    for date_of_birth in (None, "2019-10-01", "2019-10-02", "2019-10-03", "2019-10-04"):
        Rat.insert((
            (random.choice(lab_names), random.randint(1, 1_000_000), date_of_birth) 
                   for i in range(100_000)), skip_duplicates=True)

In [16]:
populate_rats()

In [17]:
Rat()

lab_name,rat_id lab-specific ID,date_of_birth
Cajal,3,2019-10-02
Cajal,16,2019-10-01
Cajal,21,2019-10-02
Cajal,29,2019-10-01
Cajal,30,None
Cajal,40,2019-10-01
Cajal,49,2019-10-01
Cajal,59,2019-10-03
Cajal,65,2019-10-03
Cajal,68,2019-10-04


Note that dispite the fact that `rat_id` is in the index, search by `rat_id` alone are not helped by the index because it is not first in the index. This is similar to search for a word in a dictionary that orders words alphabetically. Searching by the first letters of a word is easy but searching by the last few letters of a word requires scanning the whole dictionary.

In this table, the primary key is a unique index on the combination `(lab_id, rat_id)`. Therefore searches on these attributes or on `lab_id` alone are fast. But this index cannot help searches on `rat_id` alone:

In [18]:
%%timeit -n2 -r10

# inefficient!  Requires full table scan.
(Rat() & {'rat_id': 300}).fetch()

93.2 ms ± 7.87 ms per loop (mean ± std. dev. of 10 runs, 2 loops each)


In [19]:
%%timeit -n2 -r10

# efficient! Uses the primary key
(Rat() & {'rat_id': 300, 'lab_name': 'Cajal'}).fetch()

3.37 ms ± 571 µs per loop (mean ± std. dev. of 10 runs, 2 loops each)


In [20]:
%%timeit -n2 -r10

# inefficient! Requires a full table scan
len(Rat & {'rat_id': 500})

86.6 ms ± 6.96 ms per loop (mean ± std. dev. of 10 runs, 2 loops each)


Pattern searches in strings can benefit from an index when the starting characters are specified.

In [21]:
%%timeit -n2 -r10

# efficient! Uses the primary key
len(Rat & 'lab_name LIKE "Caj%"')

33.6 ms ± 1.75 ms per loop (mean ± std. dev. of 10 runs, 2 loops each)


In [22]:
%%timeit -n2 -r10

# inefficient! requires a full table scan
len(Rat & 'lab_name LIKE "%jal"')

88.9 ms ± 7.96 ms per loop (mean ± std. dev. of 10 runs, 2 loops each)


Similarly, searching by the date requires an inefficient full-table scan:

In [23]:
%%timeit -n2 -r10

len(Rat & 'date_of_birth > "2019-10-02"')

122 ms ± 11 ms per loop (mean ± std. dev. of 10 runs, 2 loops each)


To speed up searches by the `rat_id` and `date_of_birth`, we can explicit indexes to `Rat`:

In [24]:
Rat.drop()

`test_indexes`.`rat` (469640 tuples)
Proceed? [yes, No]: yes
Tables dropped.  Restart kernel.


In [25]:
@schema
class Rat(dj.Manual):
    definition = """
    lab_name : char(16) 
    rat_id : int unsigned # lab-specific ID
    ---
    date_of_birth = null : date

    index(rat_id)
    index(date_of_birth)
    """

In [26]:
populate_rats()

In [27]:
%%timeit -n2 -r10

# efficient!  uses index on rat_id
(Rat() & {'rat_id': 300}).fetch()

3.49 ms ± 446 µs per loop (mean ± std. dev. of 10 runs, 2 loops each)


In [28]:
%%timeit -n2 -r10

# efficient! uses index on date_of_birth
len(Rat & 'date_of_birth > "2019-10-02"')

56.2 ms ± 3.63 ms per loop (mean ± std. dev. of 10 runs, 2 loops each)


#### Quiz: How many indexes does the table `Rat` have?

In [29]:
Rat.describe();

lab_name             : char(16)                     
rat_id               : int unsigned                 # lab-specific ID
---
date_of_birth=null   : date                         
INDEX (rat_id)
INDEX (date_of_birth)



#### Answer
Three: `(lab_name, rat_id)` -- primary index, `(rat_id)`, and `(date_of_birth)`